#### Mini-Project # 4

This project will focus on suitability analysis with raster data. Your tasks will be both conceptual-level and technical. 

At the conceptual level, you will define a suitability model of your choice, for an area of your choice. See https://en.wikipedia.org/wiki/Suitability_model for a brief description of what a suitability model is. For example, you may be looking for best areas for community gardens: these are often in underutilized land in residential land uses, with good soils, accessible (not steep slope), etc. So you would be looking for areas with a specific type of land use/land cover, with an appropriate range of values of slope, etc. You may build additional criteria based on a range of precipitaiton values, whether the area is affected by wildfires, has mild temperatures, low levels of soil erosion, has vegeration (eg derived from satellite imagery using NDVI), etc. Feel free to use the imagery layers we explored or mentioned during the raster-focused lectures. One of the cells in the last raster notebook contained a list of about a dozen such layers available through AGOL - but feel free to find more. You can use any two of the map combination techniques discussed in the lecture on Thu 5/9. You should clearly identify the map combination technique you use, and discuss any uncertainty issues associated with these specific techniques. 

As the outcome of this part, you will need to a) describe the suitability model you want to develop; b) identify the raster data layers you will use; and c) describe two of the map combination techniques you will use to derive the two suitablity maps, and their pros and cons. In your model you will need to combine at least 3 raster data layers.

The second part will involve implementing your suitability model using arcgis raster functions. For this exercise, we'll only use local functions, and possibly focal_statistics. This is the current state of implementation of raster functions accessible through python API and in ArcGIS Online. Examples of what works are in the lecture notebooks. Look throufgh the last notebook carefully, as there are a few extra cells not shown in the lecture, which demonstrate *working* map combination examples for different types of layers. 

Be creative - but I'd rather not have you wander into blind alleys too much as you look for a declared function and it is simply not there. Note that since some functions are associated with raster services, they depend on the version of the server a particular imagery layer is served from. See in the lecture notebooks how you can retrieve the version of API supported by each layer, and which functions are supported in each API. 

The third part will be a brief write-up comparing the two output rasters generated for your suitability model using the two map combination techniques. 

The notebook should include documentation of the steps, as usually.

### Names: Yu-Chun Chen, Yanyu Tao

### IDs: A13356506, A13961185


1. Formulate a suitability model (markdown, about 100 words)

We are looking for suitable areas in San Diego to grow avocado trees, and define this as our suitability model. After research, we found that areas with rainfall 1000-1600mm, soil pH range 6-6.5 (low erodibility), 60-85 degrees F temperature, elevation under 1000 feet, slope under 15 degrees, and no strong wind to be the best places to grow avocado trees. Thus, we will look for areas that have lower precipitation (as we can irrigate for lower precipitated areas), higher temperature, and lower erodibility. Among these conditions, temperature and precipitation are the more important factors.

In [1]:
# 2. Imports, etc.
from arcgis import GIS
from IPython.display import display
import arcgis.raster
from arcgis.raster import *
from arcgis.geocoding import geocode
from arcgis import geometry
gis = GIS(username='yuc330_UCSDOnline9')

Enter password: ········


In [ ]:
# 3. List imagery layers to be used in your model. This cell should contian layer definitions. 
# Include at least three initial raster sources. 

In [2]:
#usa precipitation
#The layer contains the annual average precipitation data for the United States
rainfall = ImageryLayer("https://landscape3.arcgis.com/arcgis/rest/services/USA_Mean_Rainfall/ImageServer", gis) 
#usa mean temperature
#The layer contains the annual average temperature data for the United States
temperature = ImageryLayer("https://landscape3.arcgis.com/arcgis/rest/services/USA_Mean_Temperature/ImageServer", gis)
#soil erodibility
#The layer contains the level of erodibility in the United States, areas are classified to six classes
soil_erosion = ImageryLayer('https://landscape11.arcgis.com/arcgis/rest/services/USA_Soils_Erodibility_Factor/ImageServer',gis)

In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# 4. Derive the area of interest (AOI) and its geometry and extent. 
# The smaller the area the better (so that you don't run into raster size limitations)

In [3]:
#find the study area geometry
from arcgis.features import FeatureLayerCollection
url = 'https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Counties_Generalized/FeatureServer'
county = FeatureLayerCollection(url)
county_lyr = county.layers[0]
study_area_query = county_lyr.query("FIPS='06073'", return_geometry=True)
study_area_geom= study_area_query.features[0].geometry
study_area_geom['spatialReference'] = study_area_query.spatial_reference

In [4]:
#find the study area extent
study_area_ex = geometry.Geometry(study_area_geom).geoextent 
tup = (('xmin',study_area_ex[0]), ('ymin', study_area_ex[1]), ('xmax',study_area_ex[2]), ('ymax', study_area_ex[3]))
study_area_extent = dict(tup)
crs = "{'latestwkid':3857, 'wkid':102100}"
study_area_extent.update({'spatialReference':{'latestwkid':3857, 'wkid':102100}})

In [5]:
study_area_extent

{'spatialReference': {'latestwkid': 3857, 'wkid': 102100},
 'xmax': -12922139.2950188,
 'xmin': -13090875.0407958,
 'ymax': 3963248.36768519,
 'ymin': 3833847.56313313}

In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

5. Name the two map combination techniques you will use to combine the data and describe their pros and cons (markdown) 

We will first convert each layer to binary layer. Then we will use (1) dominance approach, binary multiplication and (2) contributory approach, binary addition to combine layers. (1) For multiplication, the pros include that we can immedeiately filter out areas with one or more unsatisfied conditions, so that we can make sure the areas left have all conditions met. On the other side, its cons include that we can possibly filter out too much areas, and for those areas filtered out, we cannot figure out which areas are more suitable than other ones, because their values are all 0. (2) For summation, the pros include that we can rank all the areas based on how many conditions they have satisfied. The cons are that it is harder to figure out areas that have satisfied all conditions immediately.

In [ ]:
# 6. Prepare your input layers for map combination: 
# clip to AOI, remap/normalize, add color map, visualize the layers and legends. 

In [6]:
#clip each layer to San Diego area
rainfall_study = clip(raster=rainfall, geometry=study_area_geom)
temperature_study = clip(raster=temperature, geometry=study_area_geom)
ero_study = clip(raster=soil_erosion, geometry=study_area_geom)

In [7]:
#remap the rainfall layer to binary color map
clrmap_rainfall =  [[1, 192, 221, 234], [0, 13, 71, 165]]

rainfall_binary = remap(rainfall_study,
                       input_ranges=[0, 30, 30, 300],
                       output_values=[1,0], astype='U8')

rainfall_m = colormap(rainfall_binary, colormap=clrmap_rainfall)

In [8]:
#remap the temperature layer to binary color map
clrmap_temp =  [[1, 188, 54, 39], [0, 247, 170, 61]]

temp_binary = remap(temperature_study,
                       input_ranges=[-10, 15, 15, 30],
                       output_values=[0,1], astype='U8')

temp_m = colormap(temp_binary, colormap=clrmap_temp)

In [9]:
#remap the soil erodibility layer to binary color map
clrmap_ero =  [[1, 183, 157, 132], [0, 99, 54, 11]]

ero_binary = remap(ero_study,
                       input_ranges=[0, 0.2, 0.2, 0.7],
                       output_values=[1,0], astype='U8')

ero_m = colormap(ero_binary, colormap=clrmap_ero)

In [10]:
#elevation layer added to workaround in 7a and 7b
usa_elevation = ImageryLayer('https://elevation.arcgis.com/arcgis/rest/services/WorldElevation/Terrain/ImageServer',gis)
slope_lyr = slope(dem=usa_elevation, slope_type='DEGREE', z_factor=10)
slope_lyr.extent = study_area_extent
slope_study_area = clip(raster=slope_lyr, geometry=study_area_geom)

In [11]:
#remap the slope layer to binary color map
clrmap_slope =  [[1, 54, 201, 76], [0, 8, 81, 19]]

slope_binary = remap(slope_study_area,
                       input_ranges=[0, 15, 15, 90],
                       output_values=[1,0], astype='U8')

slope_m = colormap(slope_binary, colormap=clrmap_slope)

In [12]:
study_area_extent

{'spatialReference': {'latestwkid': 3857, 'wkid': 102100},
 'xmax': -12922139.2950188,
 'xmin': -13090875.0407958,
 'ymax': 3963248.36768519,
 'ymin': 3833847.56313313}

In [13]:
#map the binary rainfall layer
map_rain = gis.map()
map_rain.extent = study_area_extent
map_rain.legend=True
map_rain.add_layer(rainfall_m)
map_rain

#value 1 indicates areas having precipitation less than 30 inches a year

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [14]:
#map the binary temperature layer
map_temp = gis.map()
map_temp.extent = study_area_extent
map_temp.legend=True
map_temp.add_layer(temp_m)
map_temp

#value 1 indicates areas having average annual temperature 15 to 30 degrees C

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [15]:
#map the binary erodibility layer
map_ero = gis.map()
map_ero.extent = study_area_extent
map_ero.legend=True
map_ero.add_layer(ero_m)
map_ero

#value 1 indicates areas with lower soil erodibility

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [16]:
#map the binary slope layer
map_slope = gis.map()
map_slope.extent = study_area_extent
map_slope.legend=True
map_slope.add_layer(slope_m)
map_slope

#value 1 indicates areas with less than 15 degrees slope

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [18]:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# 7a. Generate a composite raster layer for your first map combination technique
# name of the technique (refer to lecture PPT):

In [17]:
# name of the technique (refer to lecture PPT): Exclusionary screening

#combine all binary layers by multiplication
clrmap1 = [[1, 18, 117, 12], [0, 0, 7, 0]]
tech1 = raster_calculator(rasters = [rainfall_m, temp_m, ero_m], input_names = ["a","b","c"], expression = "a * b * c", extent_type='FirstOf', cellsize_type='FirstOf', astype='UNKNOWN')
cl1 = colormap(tech1, colormap=clrmap1)

#map the combined layer
map1 = gis.map()
map1.extent = study_area_extent
map1.legend=True
map1.add_layer(cl1)
map1

#was unable to show the layer

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [24]:
#checking layer provenece
cl1._fn

{'rasterFunction': 'Colormap',
 'rasterFunctionArguments': {'Colormap': [[1, 18, 117, 12], [0, 0, 7, 0]],
  'Raster': {'outputPixelType': 'UNKNOWN',
   'rasterFunction': 'RasterCalculator',
   'rasterFunctionArguments': {'CellsizeType': 0,
    'Expression': 'a * b * c',
    'ExtentType': 0,
    'InputNames': ['a', 'b', 'c'],
    'Rasters': [{'rasterFunction': 'Colormap',
      'rasterFunctionArguments': {'Colormap': [[1, 192, 221, 234],
        [0, 13, 71, 165]],
       'Raster': {'outputPixelType': 'U8',
        'rasterFunction': 'Remap',
        'rasterFunctionArguments': {'InputRanges': [0, 30, 30, 300],
         'OutputValues': [1, 0],
         'Raster': {'rasterFunction': 'Clip',
          'rasterFunctionArguments': {'ClipType': 1,
           'ClippingGeometry': {'rings': [[[-13038640.2378045,
               3833847.56313313],
              [-13046623.4026057, 3857991.25757854],
              [-13037806.2894249, 3842709.77649226],
              [-13038242.9630011, 3852764.625192],

In [26]:
#try working around with adding slope condition by adding slope layer and use it as the first in computation

#combine all binary layers by multiplication
clrmap1_w = [[1, 18, 117, 12], [0, 0, 7, 0]]
tech1_w = raster_calculator(rasters = [slope_m, rainfall_m, temp_m, ero_m], input_names = ["a","b","c","d"], expression = "a * b * c * d", extent_type='FirstOf', cellsize_type='FirstOf', astype='UNKNOWN')
cl1_w = colormap(tech1_w, colormap=clrmap1_w)

#map the combined layer
map1_w = gis.map()
map1_w.extent = study_area_extent
map1_w.legend=True
map1_w.add_layer(cl1_w)
map1_w

#This time the layer is shown on the map
#value 1 and the green color indicates the best suitable areas to grow avocado trees

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [27]:
#checking layer provenece
cl1_w._fn

{'rasterFunction': 'Colormap',
 'rasterFunctionArguments': {'Colormap': [[1, 18, 117, 12], [0, 0, 7, 0]],
  'Raster': {'outputPixelType': 'UNKNOWN',
   'rasterFunction': 'RasterCalculator',
   'rasterFunctionArguments': {'CellsizeType': 0,
    'Expression': 'a * b * c * d',
    'ExtentType': 0,
    'InputNames': ['a', 'b', 'c', 'd'],
    'Rasters': [{'rasterFunction': 'Colormap',
      'rasterFunctionArguments': {'Colormap': [[1, 54, 201, 76],
        [0, 8, 81, 19]],
       'Raster': {'outputPixelType': 'U8',
        'rasterFunction': 'Remap',
        'rasterFunctionArguments': {'InputRanges': [0, 15, 15, 90],
         'OutputValues': [1, 0],
         'Raster': {'rasterFunction': 'Clip',
          'rasterFunctionArguments': {'ClipType': 1,
           'ClippingGeometry': {'rings': [[[-13038640.2378045,
               3833847.56313313],
              [-13046623.4026057, 3857991.25757854],
              [-13037806.2894249, 3842709.77649226],
              [-13038242.9630011, 3852764.6251

In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# 7b. Generate a composite raster layer for your second map combination technique
# name of the technique (refer to lecture PPT):

In [19]:
# name of the technique (refer to lecture PPT): Voting Tabulation

#combine all binary layers by addition
clrmap2 = [[3, 247, 142, 22], [2, 247, 166, 74], [1, 244, 189, 127], [0, 244, 227, 208]]
tech2 = raster_calculator(rasters = [rainfall_m, temp_m, ero_m], input_names = ["a","b","c"], expression = "a + b + c", extent_type='FirstOf', cellsize_type='FirstOf', astype='UNKNOWN')
cl2 = colormap(tech2, colormap=clrmap2)

#map the combined layer
map2 = gis.map()
map2.extent = study_area_extent
map2.legend=True
map2.add_layer(cl2)
map2

#was also unable to show the lawer

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [28]:
#checking layer provenece
cl2._fn

{'rasterFunction': 'Colormap',
 'rasterFunctionArguments': {'Colormap': [[3, 247, 142, 22],
   [2, 247, 166, 74],
   [1, 244, 189, 127],
   [0, 244, 227, 208]],
  'Raster': {'outputPixelType': 'UNKNOWN',
   'rasterFunction': 'RasterCalculator',
   'rasterFunctionArguments': {'CellsizeType': 0,
    'Expression': 'a + b + c',
    'ExtentType': 0,
    'InputNames': ['a', 'b', 'c'],
    'Rasters': [{'rasterFunction': 'Colormap',
      'rasterFunctionArguments': {'Colormap': [[1, 192, 221, 234],
        [0, 13, 71, 165]],
       'Raster': {'outputPixelType': 'U8',
        'rasterFunction': 'Remap',
        'rasterFunctionArguments': {'InputRanges': [0, 30, 30, 300],
         'OutputValues': [1, 0],
         'Raster': {'rasterFunction': 'Clip',
          'rasterFunctionArguments': {'ClipType': 1,
           'ClippingGeometry': {'rings': [[[-13038640.2378045,
               3833847.56313313],
              [-13046623.4026057, 3857991.25757854],
              [-13037806.2894249, 3842709.776492

In [20]:
#try working around with using slope layer as the first in computation

#combine all binary layers by multiplication
clrmap2_w = [[4, 0, 5, 12], [3, 30, 81, 153], [2, 73, 138, 229], [1, 124, 170, 234], [0, 206, 220, 239]]
tech2_w = raster_calculator(rasters = [slope_m, rainfall_m, temp_m, ero_m], input_names = ["a","b","c","d"], expression = "a + b + c + d", extent_type='FirstOf', cellsize_type='FirstOf', astype='UNKNOWN')
cl2_w = colormap(tech2_w, colormap=clrmap2_w)

#map the combined layer
map2_w = gis.map()
map2_w.extent = study_area_extent
map2_w.legend=True
map2_w.add_layer(cl2_w)
map2_w


#layer is shown on the map
#Greater values indicate areas with more conditions satisfied to grow avocado trees
#Greater values are represented by darker color

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [29]:
#checking layer provenece
cl2_w._fn

{'rasterFunction': 'Colormap',
 'rasterFunctionArguments': {'Colormap': [[4, 0, 5, 12],
   [3, 30, 81, 153],
   [2, 73, 138, 229],
   [1, 124, 170, 234],
   [0, 206, 220, 239]],
  'Raster': {'outputPixelType': 'UNKNOWN',
   'rasterFunction': 'RasterCalculator',
   'rasterFunctionArguments': {'CellsizeType': 0,
    'Expression': 'a + b + c + d',
    'ExtentType': 0,
    'InputNames': ['a', 'b', 'c', 'd'],
    'Rasters': [{'rasterFunction': 'Colormap',
      'rasterFunctionArguments': {'Colormap': [[1, 54, 201, 76],
        [0, 8, 81, 19]],
       'Raster': {'outputPixelType': 'U8',
        'rasterFunction': 'Remap',
        'rasterFunctionArguments': {'InputRanges': [0, 15, 15, 90],
         'OutputValues': [1, 0],
         'Raster': {'rasterFunction': 'Clip',
          'rasterFunctionArguments': {'ClipType': 1,
           'ClippingGeometry': {'rings': [[[-13038640.2378045,
               3833847.56313313],
              [-13046623.4026057, 3857991.25757854],
              [-13037806.28

In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# 8. Compare the results, and describe how different map combination techniques resulted in different outputs (or not.)
# 

   As we are unable to visualize the layers combination we perform, we will compare outputs as we expect. We expect that the map with the multiplication layer to only have San Diego areas classified to two color classes, either all conditions of precipitation, temperature, and soil erodibility are satisfied, or at least one conditions are not met. On the other hand, we expect that the map with the addition layer to have San Diego areas classified to four color classes, areas that have all three conditions met, areas that have two of the conditions met, areas that have only one condition met, and areas having no condition met. In the latter map, we can better visualize the rank of each area when it comes to the suitability of growing avocado trees. In the former map, we immediately recognize those areas that are the most suitable to grow avocado trees.  
      
   These expectations are confirmed after we perform the workaround method, using the slope/elevation layer in the first of raster computation. We see that by multiplication, the map is classified to two colors. Areas with darker colors are those with value 1, and thus the areas that satisfy all conditions for growing avocado trees. In the map with layer of raster addition, we see that areas are classified to five classes. Areas with darker colors have greater values, and thus more suitable to grow avocado trees.  
      
   In the latter map, we have to zoom in the map to better differentiate between areas as there are many colors involved, while we can immediately point out the areas in the former map which are best for growing avocado trees.

In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

## Issue Report

In [21]:
#unable to call compute_histograms on the combined layers (all three layers combined)
tech1.compute_histograms(geometry=study_area_geom)
tech2.compute_histograms(geometry=study_area_geom)

Unable to complete operation.
Authentication token required. (status code 499).


RuntimeError: Unable to complete operation.
Authentication token required. (status code 499).
(Error Code: 400)

In [22]:
#Unable to show the combined layers on the map if not using slope/elevation layer in the first of computation
map1

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), legend=True, ready=True)

In [23]:
map2

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), legend=True, ready=True)

In [ ]:
# Please let us know how much time you spent on this project, in hours: 
# (we will only examine distributions and won't look at individual responses)
assignment_timespent = 6
extracredit_timespent = 1